# Demo 2: Naive Bayes and DataStax Analytics
------
<img src="images/drinkWine.jpeg" width="300" height="500">


#### Dataset: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

## What are we trying to learn from this dataset? 

# QUESTION:  Can Naive Bayes be used to classify a wine’s rating score by its attributes?

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt

In [21]:
import pandas
import cassandra
import pyspark
import re
import os
import random
from random import randint, randrange
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

#### Helper function to have nicer formatting of Spark DataFrames

In [22]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

# DataStax Enterprise Analytics
<img src="images/dselogo.png" width="400" height="200">

## Creating Tables and Loading Tables

### Connect to DSE Analytics Cluster

In [23]:
from cassandra.cluster import Cluster

cluster = Cluster(['dse'])
session = cluster.connect()

### Create Demo Keyspace 

In [24]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS accelerate 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

### Set keyspace 

In [25]:
session.set_keyspace('accelerate')

### Create table called `wines`. Our PRIMARY will be a unique key (wineid) we generate for each row.  This will have two datasets "white" and "red"

In [27]:
query = "CREATE TABLE IF NOT EXISTS wines \
                                   (wineid int, fixedAcidity float, volatileAcidity float, citricAcid float, sugar float, \
                                   chlorides float, freeSulfur float, totalSulfur float, density float, ph float, \
                                   sulphates float, alcohol float, quality float, \
                                   PRIMARY KEY (wineid))"
session.execute(query)

### What do these of these 12 columns represent: 

* **Fixed acidity**
* **Volatile acidity**
* **Citric Acid**
* **Residual Sugar** 
* **Chlorides**
* **Free sulfur dioxide**     
* **Total sulfur dioxide**
* **Density** 
* **pH**
* **Sulphates**
* **Alcohol**
* **Quality**

### Load 2 Wine Dataset -- White and Red
<img src="images/whiteAndRed.jpeg" width="300" height="300">

### Load Wine datasets from CSV file (winequality-red.csv winequality-white.csv)
* No clean up was requried! How nice :)

#### Insert all the Wine Data into the DSE table `wines`

In [28]:
fileName = 'data/winequality-red.csv'
input_file = open(fileName, 'r')
i = 1
for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wines (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1

fileName = 'data/winequality-white.csv'
input_file = open(fileName, 'r')

for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wines (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1
    

## Machine Learning with DSE Analytics and Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

#### Create a spark session that is connected to DSE. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [29]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()

wineDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="wines", keyspace="accelerate").load()

print ("Table Wine Row Count: ")
print (wineDF.count())

Table Wine Row Count: 
6497


In [30]:
showDF(wineDF)

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity
0,4317,10.8,0.046,0.29,0.99518,6.8,59.0,3.20,6.0,10.4,0.40,143.0,0.16
1,3372,10.9,0.059,0.26,0.99550,7.8,32.0,3.04,6.0,9.5,0.43,178.0,0.40
2,1584,9.8,0.074,0.29,0.99578,6.2,32.0,3.33,5.0,2.1,0.62,98.0,0.46
3,4830,9.4,0.056,0.57,0.99548,6.7,60.0,2.96,6.0,6.6,0.43,150.0,0.13
4,2731,9.7,0.047,0.34,0.99440,6.9,24.0,3.20,6.0,4.0,0.52,128.0,0.23


#### Let's filter out only wines that have been rated 6.0 or higher and create a new dataframe with that information 

In [31]:
wine6DF = wineDF.filter("quality > 5")
showDF(wine6DF)

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity
0,4317,10.8,0.046,0.29,0.99518,6.8,59.0,3.20,6.0,10.4,0.40,143.0,0.16
1,3372,10.9,0.059,0.26,0.99550,7.8,32.0,3.04,6.0,9.5,0.43,178.0,0.40
2,4830,9.4,0.056,0.57,0.99548,6.7,60.0,2.96,6.0,6.6,0.43,150.0,0.13
3,2731,9.7,0.047,0.34,0.99440,6.9,24.0,3.20,6.0,4.0,0.52,128.0,0.23
4,769,9.7,0.082,0.02,0.99744,7.1,24.0,3.55,6.0,2.3,0.53,94.0,0.59


#### Create Vector with all elements of the wine 

In [32]:
assembler = VectorAssembler(
    inputCols=['alcohol', 'chlorides', 'citricacid', 'density', 'fixedacidity', 'ph', 'freesulfur', 'sugar', 'sulphates', 'totalsulfur', 'volatileacidity'],
    outputCol='features')

trainingData = assembler.transform(wine6DF)

labelIndexer = StringIndexer(inputCol="quality", outputCol="label", handleInvalid='keep')
trainingData1 = labelIndexer.fit(trainingData).transform(trainingData)

showDF(trainingData1)
print(trainingData1.count())

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label
0,4317,10.8,0.046,0.29,0.99518,6.8,59.0,3.20,6.0,10.4,0.40,143.0,0.16,"[10.800000190734863, 0.04600000008940697, 0.28...",0.0
1,3372,10.9,0.059,0.26,0.99550,7.8,32.0,3.04,6.0,9.5,0.43,178.0,0.40,"[10.899999618530273, 0.05900000035762787, 0.25...",0.0
2,4830,9.4,0.056,0.57,0.99548,6.7,60.0,2.96,6.0,6.6,0.43,150.0,0.13,"[9.399999618530273, 0.0560000017285347, 0.5699...",0.0
3,2731,9.7,0.047,0.34,0.99440,6.9,24.0,3.20,6.0,4.0,0.52,128.0,0.23,"[9.699999809265137, 0.04699999839067459, 0.340...",0.0
4,769,9.7,0.082,0.02,0.99744,7.1,24.0,3.55,6.0,2.3,0.53,94.0,0.59,"[9.699999809265137, 0.0820000022649765, 0.0199...",0.0


4113


### We will be training a model with Naive Bays, and because of this we need to split up our dataset in to a training and test set. Will split 80/20. 

In [33]:
# Split the data into train and test
splits = trainingData1.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

print ("Train Dataframe Row Count: ")
print (train.count())
print ("Test Dataframe Row Count: ")
print (test.count())

Train Dataframe Row Count: 
3366
Test Dataframe Row Count: 
747


### Now it's time to to use NaiveBayes. We will train the model, then use that model with out testing data to get our predictions. 
https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#naive-bayes

In [39]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

predictions = model.transform(test)
#predictions.show()
print (predictions.count())
showDF(predictions)

747


,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label,rawPrediction,probability,prediction
0,8,10.0,0.065,0.00,0.9946,7.3,15.0,3.39,7.0,1.2,0.47,21.0,0.650,"[10.0, 0.06499999761581421, 0.0, 0.99459999799...",1.0,"[-116.61332062455122, -116.11569707646935, -11...","[0.3624807686165096, 0.5962112012919782, 0.038...",1.0
1,30,9.8,0.082,0.00,0.9964,7.8,8.0,3.38,6.0,2.0,0.59,16.0,0.645,"[9.800000190734863, 0.0820000022649765, 0.0, 0...",0.0,"[-107.00826876792438, -106.69331316801713, -10...","[0.4109511317563823, 0.5630846130734202, 0.023...",1.0
2,43,10.5,0.332,0.20,0.9968,7.5,8.0,3.21,6.0,2.6,0.90,14.0,0.490,"[10.5, 0.3319999873638153, 0.20000000298023224...",0.0,"[-112.66442184314677, -112.30188509993125, -11...","[0.3992810255077367, 0.5737548627763441, 0.022...",1.0
3,129,10.5,0.065,0.16,0.9962,8.0,3.0,3.42,7.0,1.8,0.92,16.0,0.590,"[10.5, 0.06499999761581421, 0.1599999964237213...",1.0,"[-102.8833290398864, -102.54742453805858, -106...","[0.40916428022145745, 0.5725048690933248, 0.01...",1.0
4,221,9.4,0.082,0.37,0.9964,7.8,24.0,3.34,6.0,2.0,0.59,58.0,0.340,"[9.399999618530273, 0.0820000022649765, 0.3700...",0.0,"[-151.32966193268658, -151.34435617974762, -15...","[0.4777992886623408, 0.47082971956014313, 0.04...",0.0


In [40]:
showDF(predictions.select("quality", "label", "prediction", "probability"))

,quality,label,prediction,probability
0,7.0,1.0,1.0,"[0.3624807686165096, 0.5962112012919782, 0.038..."
1,6.0,0.0,1.0,"[0.4109511317563823, 0.5630846130734202, 0.023..."
2,6.0,0.0,1.0,"[0.3992810255077367, 0.5737548627763441, 0.022..."
3,7.0,1.0,1.0,"[0.40916428022145745, 0.5725048690933248, 0.01..."
4,6.0,0.0,0.0,"[0.4777992886623408, 0.47082971956014313, 0.04..."


### We can now use the MutliclassClassifciationEvaluator to evalute the accurancy of our predictions. 

In [41]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.6010709504685409


In [26]:
session.execute("""drop table wines""")